In [ ]:
import os
import openai
from openai import OpenAI

HOME = os.environ.get("HOME")
CAFILE = HOME + "/cacert.pem"
API = "https://patagonia-aigen-srv:8080/llm-large/v1"

In [ ]:
def api_config(api):
    # Modify OpenAI's API key and API base to use vLLM's API server.
    openai_api_key = "EMPTY"
    openai_api_base = api
    os.environ["SSL_CERT_FILE"] = CAFILE

    print("[INFO] API:" , openai_api_base)

    client = OpenAI(
        api_key=openai_api_key,
        base_url=openai_api_base,
    )

    # Checking availlable model
    models = client.models.list()
    model = models.data[0].id
    print(models)
    print("[INFO] Model:" , model,"\n")
    return client, model

In [ ]:
client, model = api_config(API)

[INFO] API: https://patagonia-aigen-srv:8080/llm-large/v1
SyncPage[Model](data=[Model(id='/app/models/mistralai/Mixtral-8x7B-Instruct-v0.1', created=1721208125, object='model', owned_by='vllm', root='/app/models/mistralai/Mixtral-8x7B-Instruct-v0.1', parent=None, permission=[{'id': 'modelperm-774fd77d41fe439eb49444f7365649c0', 'object': 'model_permission', 'created': 1721208125, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}])], object='list')
[INFO] Model: /app/models/mistralai/Mixtral-8x7B-Instruct-v0.1 



In [ ]:
history = [{"role": "user", "content": "siema"}]
response = client.chat.completions.create(
    messages=history,
    model=model,
    temperature=0
)

In [ ]:
for choice in response.choices:
    message = choice.message
    if message.role == 'assistant':
        print(message.content)
#     print(message.__dict__)
#     print(message.content)
#     print(message.role)


 Hello! It seems like you've typed "siema" which might be a typo or a name. If you're looking for a conversation, I'm here to help!

I can assist you with any questions you have or engage in a discussion on a topic of your choice. Some popular topics include technology, science, current events, and pop culture.

If you need help with something specific, please provide more context or clarify your question. I'm here to make your experience as smooth as possible.

Let's have a great conversation!


In [ ]:
def read_txt_file(txt_filename):
    with open(txt_filename, 'r', encoding='utf-8') as file:
        content = file.read()
    return content
# text = read_txt_file('./Transkrypcja__Gildia_AI_Tech_2024.txt')

In [ ]:
text = read_txt_file("./Transkrypcja__Gildia_AI_Tech_2024.txt")

In [ ]:
# history = [
#     {"role": "user", "content": f"Proszę znajdź następujące informacje z dostarczonego tekstu:\n{text}\n" +
#                                "\n".join([f"{k}: {v}" for k, v in data_explanation.items()])}
# ]

In [ ]:
def split_text(text, max_length=1000):
    chunks = [text[i:i + max_length] for i in range(0, len(text), max_length)]
    return chunks


In [ ]:
def summarize_chunk(chunk):
    prompt = f"""
    Please summarize the main points discussed in the following text in Polish:
    {chunk}
    """
    history = [
    {"role": "user", "content": prompt}]

    response = client.chat.completions.create(
        messages=history,
        model=model,
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.7,
    )

    summary = response.choices[0].text.strip()
    return summary


In [ ]:
def summarize_text(text):
    chunks = split_text(text)
    summaries = [summarize_chunk(chunk) for chunk in chunks]
    combined_summaries = " ".join(summaries)
    final_summary = summarize_chunk(combined_summaries)
    return final_summary

In [ ]:
text_1 = read_txt_file('./Transkrypcja__Gildia_AI_Tech_2024.txt')

In [ ]:
final_summary = summarize_text(text_1)
print(final_summary)

AttributeError: 'Choice' object has no attribute 'text'

### proba

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import openai

In [ ]:
embed = OpenAIEmbeddings(
#     model = 'text-embedding-ada-002',
    model=model)
#     openai_api_key = "EMPTY"
#     openai_api_base = api)

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
# Load a single document using LangChain's PDF Loader
loader = PyPDFLoader(file)
doc = loader.load()

NameError: name 'file' is not defined

In [ ]:
# split documents into text and embeddings
text_splitter = RecursiveCharacterTextSplitter(
   chunk_size=1000,
   chunk_overlap=150
)

chunks = text_splitter.split_documents(document)

In [ ]:
# Create an array of text to embed
context_array = []
for i, row in enumerate(chunks):
  context_array.append(row.page_content)

# Submit array to OpenAI, which will return a list of embeddings
# calculated from the input array of text chunks
emb_vectors = embed.embed_documents(context_array)

NameError: name 'chunks' is not defined

In [ ]:
import os
import openai

# Set environment variables and constants
HOME = os.environ.get("HOME")
CAFILE = HOME + "/cacert.pem"
API = "https://patagonia-aigen-srv:8080/llm-large/v1"
CHUNK_SIZE = 1024  # Define the chunk size based on the model's maximum token limit

def api_config(api):
    # Modify OpenAI's API key and API base to use vLLM's API server.
    openai.api_key = "EMPTY"
    openai.api_base = api
    os.environ["SSL_CERT_FILE"] = CAFILE

    print("[INFO] API:", openai.api_base)

    # Checking available models
    models = openai.Model.list()
    model = models['data'][0]['id']
    print(models)
    print("[INFO] Model:", model, "\n")
    return model

# Configure API and get the model
model = api_config(API)

# Function to generate a response from the model
def generate_response(history):
    response = openai.ChatCompletion.create(
        model=model,
        messages=history,
        temperature=0
    )
    return response.choices[0].message['content']

# Function to read and split the text into chunks
def read_and_split_text(file_path, chunk_size):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Function to summarize the text chunks
def summarize_text_chunks(chunks):
    summaries = []
    for chunk in chunks:
        history = [{"role": "user", "content": f"Proszę podsumuj następujący tekst:\n{chunk}"}]
        summary = generate_response(history)
        summaries.append(summary)
    return summaries

# Function to combine and refine summaries
def refine_summary(summaries):
    combined_summary = " ".join(summaries)
    history = [{"role": "user", "content": f"Proszę podsumuj następujące podsumowania:\n{combined_summary}"}]
    refined_summary = generate_response(history)
    return refined_summary

# Main function to handle the summarization process
def summarize_long_text(file_path):
    chunks = read_and_split_text(file_path, CHUNK_SIZE)
    summaries = summarize_text_chunks(chunks)
    final_summary = refine_summary(summaries)
    return final_summary

# Example usage
file_path = "path_to_your_file.txt"
final_summary = summarize_long_text(file_path)
print("Final Summary:")
print(final_summary)


In [ ]:
import os
import openai

# Set environment variables and constants
HOME = os.environ.get("HOME")
CAFILE = HOME + "/cacert.pem"
API = "https://patagonia-aigen-srv:8080/llm-large/v1"
CHUNK_SIZE = 1024  # Define the chunk size based on the model's maximum token limit

def api_config(api):
    # Modify OpenAI's API key and API base to use vLLM's API server.
    openai.api_key = "EMPTY"
    openai.api_base = api
    os.environ["SSL_CERT_FILE"] = CAFILE

    print("[INFO] API:", openai.api_base)

    # Checking available models
    models = openai.Model.list()
    model = models['data'][0]['id']
    print(models)
    print("[INFO] Model:", model, "\n")
    return model

# Configure API and get the model
model = api_config(API)

# Function to generate a response from the model
def generate_response(history):
    response = openai.ChatCompletion.create(
        model=model,
        messages=history,
        temperature=0.7
    )
    return response.choices[0].message['content']

# Function to split text into chunks
def split_text(text, chunk_size=CHUNK_SIZE):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Function to summarize a chunk of text
def summarize_chunk(chunk):
    prompt = f"Proszę podsumuj następujący tekst:\n{chunk}"
    history = [{"role": "user", "content": prompt}]
    summary = generate_response(history)
    return summary

# Function to summarize the entire text
def summarize_text(text):
    chunks = split_text(text)
    summaries = [summarize_chunk(chunk) for chunk in chunks]
    combined_summaries = " ".join(summaries)
    final_summary = summarize_chunk(combined_summaries)
    return final_summary

# Example usage
file_path = "path_to_your_file.txt"
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

final_summary = summarize_text(text)
print("Final Summary:")
print(final_summary)
